# 조회에 필요한 데이터 불러오기

In [14]:
from tqdm import tqdm
import pandas as pd
from glob import glob

In [15]:
files = sorted(glob('./data/*.csv'))
date_cols = ['date']
data_all = pd.DataFrame()
for file in tqdm(files):
    df_day = pd.read_csv(file, parse_dates=date_cols, usecols=['press', 'title', 'date', 'link']) # date 컬럼 datetime 형식으로 불러오기
    data_all = pd.concat([data_all, df_day])
data_all.reset_index(drop=True, inplace=True)

100%|████████████████████████████████████████████████████████████████████████████████| 730/730 [01:57<00:00,  6.23it/s]


In [24]:
data_all

,press,title,date,link
0,중앙일보,프로배구 2·3일 경기 연기.. 중계방송 스태프 코로나19 확진,2021-01-01,https://v.daum.net/v/20210101233343622
1,세계일보,"""산에서 커피 팔고 싶어"" 조두순의 소박한 꿈 이뤄질까? 시민들 '불안'",2021-01-01,https://v.daum.net/v/20210101233139613
2,서울신문,확진 1000명 목전 두고..秋 '구치소 집단감염' 첫 사과(종합),2021-01-01,https://v.daum.net/v/20210101233101608
3,국민일보,문건 유출? 미루기?.. 벌써 다 퍼진 '2.5단계 연장안',2021-01-01,https://v.daum.net/v/20210101231448533
4,뉴스1,'해변 폐쇄·드론 감시' 강릉 해맞이객 90% 줄어..그래도 1만6천명,2021-01-01,https://v.daum.net/v/20210101231238518
...,...,...,...,...
1411408,세계일보,"한국남동발전, 8년째 취약계층 평생학습 지원",2022-12-31,https://v.daum.net/v/20221231010122552
1411409,세계일보,"KIS제주, 하버드대 2년 연속 합격생 배출",2022-12-31,https://v.daum.net/v/20221231010117551
1411410,한국일보,늘어나는 방음터널... 번지면 대형 화재인데 방염 규정은 전무,2022-12-31,https://v.daum.net/v/20221231001003411
1411411,국민일보,[핫인픽] #배달할증료 #한파·폭설 출근길 #메시 지폐,2022-12-31,https://v.daum.net/v/20221231000423387


# 전체 데이터 유사도 추천

## 모델 불러오기

In [16]:
import pickle

In [17]:
with open("./model/pickled_tfidf_vec.bin", "rb") as f:
    tfidf_vec = pickle.load(f)

In [18]:
with open("./model/pickled_tfidf_dtm.bin", "rb") as f:
    tfidf_dtm = pickle.load(f)

## 형태소 추출

In [19]:
def string_tokenizer(content):
    import pandas as pd
    from konlpy.tag import Okt
    okt = Okt()
    
    content = pd.Series(content)
    # 한글, 숫자, 공백만 남기기
    content = content.str.replace('[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', regex=True)
    token_content = okt.pos(content[0], norm=True, stem=True)
    return token_content

In [20]:
except_list = ['Exclamation', 'Josa', 'KoreanParticle', 'Determiner',  'Eomi', 'Suffix',  'VerbPrefix', 'PreEomi']
include_list = ['Verb',  'Noun']

In [21]:
# 230207 기현
def tag_except_list(except_list, token_content):
    filtered_list= []
    for tag in token_content :
        if tag[1] in except_list:
            continue
        filtered_list.append(tag[0])
    content = " ".join(filtered_list)
    return content

In [22]:
# 230207 기현
def tag_include_list(include_list, token_content):
    filtered_list=[]
    for tag in token_content :
        if tag[1] in include_list:
            filtered_list.append(tag[0])

    content = " ".join(filtered_list)
    return content

## 다음 url로 유사 기사 추천

In [23]:
stop_content = ["무단전재", "재배포금지","저작권자 ⓒ 서울신문사","무단복제 및 전재","무단 전재 및 재배포","제보는 카톡", "☞", "무단 전재-재배포", "▶연합뉴스 앱 지금 바로 다운받기~"]
rep_list = ['기사내용 요약']

In [24]:
def create_soup(url):
    from bs4 import BeautifulSoup
    import requests
    i = 0
    ## 요청 오류시 10번  재시도
    while i < 10 :
        try:
            res = requests.get(url)
            res.raise_for_status()
            soup = BeautifulSoup(res.content, 'html.parser', from_encoding='cp949')
            break
        except:
            i += 1
            
    return soup

In [25]:
def content_only_scraper(link):
    article_soup = create_soup(link)

    # 본문 정리
    article = article_soup.find_all('section')[1]
    content = article.find_all(True, attrs={'dmcf-ptype':'general'})
    rst = []
    for para in content:
        for tmp in para.text.split('\n'):
            if tmp.strip() != '':
                rst.append(tmp.strip())
    content = []
    for c in rst:
        for i in stop_content:
            if i in c:
                break
        else:
            for rep in rep_list:
                c = c.replace(rep, '')
            content.append(c)
    content = ' '.join(content)
    if len(content) < 200:
        return False
    
    return content

In [61]:
def get_recommend_by_url(url, n, drange, press):
    from sklearn.metrics.pairwise import cosine_similarity
    # 다음 뉴스인지 확인
    if not url.startswith('https://v.daum.net/v/'):
        return False
    
    ## 입력 처리
    # 본문 스크랩
    content = content_only_scraper(url)
    # 형태소 전처리
    token = tag_except_list(except_list, string_tokenizer(content))
    # tfidf 연산
    tfidf_content = tfidf_vec.transform([token])
    
    ## 조회 한정
    # 기간 & 언론사
    data_filtered_idx = data_all[(data_all['date'].isin(pd.date_range(drange[0], drange[1]))) & (data_all['press']==press)].index
    tfidf_dtm_filtered = tfidf_dtm[data_filtered_idx]
    cos_sim_res = cosine_similarity(tfidf_content, tfidf_dtm_filtered)
    
    # 유사도 정렬, 추출
    sim_scores = list(zip(data_filtered_idx, cos_sim_res[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores_n = sim_scores[1:n+1]
    # sim_scores_n = [sim for sim in sim_scores_n if sim[1] >= 0.6] # 유사도 60% 이상 필터
    
    # 해당하는 기사 가져오기
    article_idx = [article_dict[0] for article_dict in sim_scores_n]
    rst = data_all.iloc[article_idx][['press', 'title', 'date', 'link']]
    rst['similarity'] = [round(sim[1], 4)*100 for sim in sim_scores_n]
    
    return rst[['press', 'similarity', 'title', 'date', 'link']]

In [66]:
%%time
url = """https://v.daum.net/v/20230206162031399"""
get_recommend_by_url(url, 10, ("2021-01-01", "2022-12-31"), "뉴시스")

CPU times: total: 2.84 s
Wall time: 2.39 s


,press,similarity,title,date,link
1358639,뉴시스,48.96,'입시비리'·감찰무마' 조국 재판 오늘 결심…3년 만에 마무리 수순,2022-12-02,https://v.daum.net/v/20221202060032795
1357681,뉴시스,48.84,"'자녀 입시비리' 조국 전 장관, 1심 결심공판 출석 [뉴시스Pic]",2022-12-02,https://v.daum.net/v/20221202141500738
338902,뉴시스,48.05,"검찰 ""위조의 시간"" 지적 vs 조국 ""법률용어 써라"" 반발",2021-06-11,https://v.daum.net/v/20210611160122570
1330596,뉴시스,48.03,"檢 ""입시비리 중대범죄"", 정경심 측 ""부당 기소""…징역 2년 구형",2022-11-18,https://v.daum.net/v/20221118115238952
252388,뉴시스,47.95,"조국·정경심, 6월11일 법정 같이선다..반년만 재판 재개",2021-04-30,https://v.daum.net/v/20210430121651371
339614,뉴시스,47.82,"조국 측 ""검찰의 투망식 공소""..'감찰무마' 거듭 부인",2021-06-11,https://v.daum.net/v/20210611112445421
327314,뉴시스,47.66,"조국·정경심, 나란히 법정 선다..6개월만에 재판 재개",2021-06-06,https://v.daum.net/v/20210606050052336
1346082,뉴시스,47.49,"조국 입시비리 등 의혹, 이번주 결심…이르면 올해 1심 결론",2022-11-27,https://v.daum.net/v/20221127095726043
1357622,뉴시스,47.43,"검찰, 조국 '자녀 입시비리·감찰무마 혐의' 징역 5년 구형",2022-12-02,https://v.daum.net/v/20221202143703564
338720,뉴시스,46.68,"조국·정경심 재판에 아들·딸도 부른다..""온가족이 증인""(종합)",2021-06-11,https://v.daum.net/v/20210611171621584


## 직접 입력한 본문(문자열)로 유사 기사 추천

In [14]:
def get_recommend_by_content(content, n, drange=False):
    from sklearn.metrics.pairwise import cosine_similarity
    ## 입력 처리
    # 형태소 전처리
    token = tag_except_list(except_list, string_tokenizer(content))
    # tfidf 연산
    tfidf_content = tfidf_vec.transform([token])
    
    ## 조회 한정
    # 기간 & 언론사
    data_filtered_idx = data_all[(data_all['date'].isin(pd.date_range(drange[0], drange[1]))) & (data_all['press']==press)].index
    tfidf_dtm_filtered = tfidf_dtm[data_filtered_idx]
    cos_sim_res = cosine_similarity(tfidf_content, tfidf_dtm_filtered)
    
    # 유사도 정렬, 추출
    sim_scores = list(enumerate(cos_sim_res[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores_n = sim_scores[1:n+1]
    # sim_scores_n = [sim for sim in sim_scores_n if sim[1] >= 0.6] # 유사도 60% 이상 필터=>옵션화 가능?###############
    
    # 해당하는 기사 가져오기
    article_idx = [article_dict[0] for article_dict in sim_scores_n]
    rst = data_all.iloc[article_idx][['press', 'title', 'date', 'link']]
    rst['similarity'] = [round(sim[1], 4)*100 for sim in sim_scores_n]
        
    return rst[['press', 'similarity', 'title', 'date', 'link']]

In [ ]:
%%time
arc_content = """2011년 한국을 떠나 러시아로 귀화한 전 쇼트트랙 선수 빅토르 안(38·한국명 안현수)의 국내 복귀 시도가 일단 무산됐다. 경기도 성남시청 직장운동부 빙상팀 코치에 지원했으나, 최종 후보에 들지 못해서다.

성남시는 29일 “시청 빙상팀 코치직 채용 전형에 빅토르 안을 포함해 7명이 지원했는데, 빅토르 안은 상위 2배수 후보에 들지 않았다”고 밝혔다. 시 관계자는 “서류와 면접 심사를 통해 기술, 소통 능력 등 여러 요소를 종합해 판단했다”며 “빙상계 여론과 언론 보도 등을 통해 나오는 시각도 평가에 반영됐다”고 했다.

성남시는 지난해 12월 19일 빙상팀 코치를 뽑기 위한 채용 공고를 냈다. 빅토르 안과 베이징 동계올림픽에서 중국 대표팀을 이끈 김선태 전 감독 등 7명이 지원했다. 김 전 감독도 최종 후보에 들지 못한 것으로 전해졌다.

빅토르 안은 2006년 토리노 동계올림픽에서 3개의 금메달을 딴 한국 쇼트트랙의 간판이었다. 2011년 소속팀이었던 성남시청이 빙상팀을 해체하자 선수 생활을 이어가기 위해 러시아로 귀화했다. 2014년 소치 동계올림픽에는 러시아 선수로 나서 3관왕에 올랐다. 2018년 평창 동계올림픽 출전이 무산된 이후에는 은퇴를 선언하고 지도자로 변신했다. 2022년 베이징 동계올림픽에서는 중국 대표팀 코치로 활동했다.

성남시는 31일 빙상팀 코치 선발 결과를 발표할 예정이다."""
get_recommend_by_content(arc_content, 10)

NameError: name 'tag_except_list' is not defined

In [19]:
%%time
arc_content = """19년 간 2억원이 넘는 기부를 실천한 사람이 있다.

지난 2004년부터 매년 꾸준히 어려운 이웃을 위한 나눔을 실천, 19년간 2억 2000만원을 기부한 시민이 있어 지역사회의 귀감이 되고 있다.

코로나19가 길어져 취약 계층의 복지 수요가 늘었다고 판단한 황 대표는 올해 1월부터는 기부액을 매달 200만원으로 늘렸다.

이렇게 이어온 황 대표의 누적 기부 금액은 2억 2000만원에 달한다."""
get_recommend_by_content(arc_content, 10)

CPU times: total: 12.6 s
Wall time: 9.03 s


,press,similarity,title,date,link
1125112,뉴스1,42.95,19년 동안 2억2000만원 내 놓은 기부천사 황영희씨,2022-07-28,https://v.daum.net/v/20220728101841849
223041,중앙일보,33.62,"""소아암 아이 함께 돕자""..MZ세대는 SNS로 나눈다 [기부,부의 품격⑤]",2021-04-16,https://v.daum.net/v/20210416050051397
392752,뉴시스,32.11,"코로나 속 이웃돕기 실천..행안부, 연말에 '착한 기부자'로 포상",2021-07-07,https://v.daum.net/v/20210707120041498
832095,세계일보,31.85,10년간 자발적 기부금품 1248억원.. 매년 증가 추세,2022-02-17,https://v.daum.net/v/20220217010846900
138753,뉴시스,31.67,"박남춘 시장, '나눔e음' 서비스 업무 협약",2021-03-08,https://v.daum.net/v/20210308090019230
624504,조선일보,31.37,"기부 좀 해본 MZ세대에게 물었다 ""기부의 미래는?""",2021-11-02,https://v.daum.net/v/20211102030345961
1061465,뉴시스,30.96,"10년 익명기부 '키다리 아저씨' 박무근 미광전업 대표, 청룡봉사상 수상",2022-06-24,https://v.daum.net/v/20220624153329353
87184,뉴스1,30.93,"[영상]""밥 한 끼라도 든든하게""..'코로나 혹한기' 녹인 나눔의 온정",2021-02-10,https://v.daum.net/v/20210210074609536
641540,국민일보,30.82,"조광한 남양주시장 '선거실패, 국가실패' 수익금 기부",2021-11-11,https://v.daum.net/v/20211111181546583
21115,세계일보,30.48,울산 '사랑의 온도탑' 39일 만에 124도 달성,2021-01-11,https://v.daum.net/v/20210111030154385
